In [1]:
import numpy as np
import cv2 as cv
import pytesseract
from pytesseract import Output
import re
import math
import time

In [2]:
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

In [3]:
from flask import Flask

UPLOAD_FOLDER = 'static/uploads/'

app = Flask(__name__)
app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024

In [4]:
cal=''
carb=''
protein=''
fat=''
text=''


def check(filename):
    img = cv.imread('static/uploads/'+filename)
    c=1
    custom_config = r'-c tessedit_char_blacklist= --psm 6'
    text=pytesseract.image_to_string(img, config=custom_config)
    d = pytesseract.image_to_data(img, output_type=Output.DICT)
    n_boxes = len(d['text'])
    for i in range(n_boxes):
        if d['text'][i]=='Calories':
            cal=d['text'][i+2]
            flash("Number of calories: "+cal)
        if d['text'][i]=='Carbohydrate':
            carb=d['text'][i+1]
            flash("Amount of Carbohydrate :"+carb)
        if d['text'][i]=='Protein':
            protein=d['text'][i+1]
            flash("Grams of Proteins: "+protein)
        if d['text'][i]=='Fat':
            if c==1:
                c=c+1
                fat=d['text'][i+1]
                flash("Total Fat: "+fat)
        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
        img = cv.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

In [5]:
import os
#from app import app
import urllib.request
from flask import Flask, flash, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename

ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg', 'gif'])

def allowed_file(filename):
	return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/static/bmibmr.html')
def bmibmr():
    return render_template('bmibmr.html')

@app.route('/')
def upload_form():
	return render_template('nutrichart.html')

@app.route('/', methods=['POST'])
def upload_image():
	if 'file' not in request.files:
		flash('No file part')
		return redirect(request.url)
	file = request.files['file']
	if file.filename == '':
		flash('No image selected for uploading')
		return redirect(request.url)
	if file and allowed_file(file.filename):
		filename = secure_filename(file.filename)
		file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
		check(filename)
		return render_template('nutrichart.html', filename=filename)
	else:
		flash('Allowed image types are -> png, jpg, jpeg, gif')
		return redirect(request.url)

@app.route('/display/<filename>')
def display_image(filename):
	return redirect(url_for('static', filename='uploads/' + filename))

if __name__ == "__main__":
    app.run(port="3000")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:3000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Feb/2022 22:25:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 22:25:48] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [19/Feb/2022 22:25:49] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [19/Feb/2022 22:27:15] "POST / HTTP/1.1" 200 -
